In [13]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Activation, Dense, InputLayer , Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar100
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import itertools
import os
import shutil
import torch
import random
import glob
import matplotlib.pyplot as plt
import warnings
import tensorflow.keras.applications.efficientnet as efn
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [2]:
os.chdir(r'C:\Users\RezaHardMan\Documents\Python projects\datasets\digits')
dt_train = pd.read_csv('train.csv')
dt_test = pd.read_csv('test.csv')
dt_sample = pd.read_csv('sample_submission.csv')
df_train = pd.DataFrame(dt_train)
df_test = pd.DataFrame(dt_test)
df_sample = pd.DataFrame(dt_sample)

In [3]:
dt_train.head(20)

#we have inputs and targets for training in this file.

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
dt_test.head(20)
#we have inputs for test but we must read targets from dt_sample file.

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
dt_sample
# we have test targets here.

,ImageId,Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
27995,27996,0
27996,27997,0
27997,27998,0
27998,27999,0


In [6]:
X_train = dt_train.drop('label', axis=1)
X_test = dt_test
y_train = dt_train['label']


In [7]:
X_train = X_train.values.reshape(-1,28,28,1)
X_test = X_test.values.reshape(-1,28,28,1)
y_train = to_categorical(y_train, num_classes = 10)

In [48]:
datagen = ImageDataGenerator(
        featurewise_center=True,  
        samplewise_center=True,  
        featurewise_std_normalization=True,  
        samplewise_std_normalization=True,   
        rotation_range=10,  
        zoom_range = 0.1,
        width_shift_range=0.1,  
        height_shift_range=0.1,
        validation_split=0.1
)


In [30]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))


In [46]:
model2 = Sequential([
    Conv2D(filters=32, kernel_size=(5, 5), activation='relu', padding = 'same', input_shape=(28,28,1)),
    Conv2D(filters=32, kernel_size=(5, 5), activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPool2D(pool_size=(2, 2)),
    
    
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPool2D(pool_size=(2, 2), strides=(2,2)),    
    Dropout(0.1),
    
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding = 'same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding = 'same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPool2D(pool_size=(2, 2), strides=(2,2)),
    Dropout(0.2),
    
    Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding = 'same'),
    Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding = 'same'),
    Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPool2D(pool_size=(2, 2), strides=(2,2)),
    Dropout(0.2),
    
    Flatten(),
    Dense(1024, activation = 'relu'),
    Dropout(0.5),

    Dense(units=10, activation='softmax')
])

print(model2.summary())
model2.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
filepath1="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
filepath2 = "best_weights.hdf5"
checkpoint1 = ModelCheckpoint(filepath1, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
checkpoint2 = ModelCheckpoint(filepath2, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')


callbacks_list = [checkpoint1,checkpoint2]

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_331 (Conv2D)         (None, 28, 28, 32)        832       
                                                                 
 conv2d_332 (Conv2D)         (None, 28, 28, 32)        25632     
                                                                 
 batch_normalization_101 (Ba  (None, 28, 28, 32)       128       
 tchNormalization)                                               
                                                                 
 max_pooling2d_128 (MaxPooli  (None, 14, 14, 32)       0         
 ng2D)                                                           
                                                                 
 conv2d_333 (Conv2D)         (None, 14, 14, 64)        18496     
                                                                 
 conv2d_334 (Conv2D)         (None, 14, 14, 64)      

In [52]:
TRAIN_STEPS_PER_EPOCH = np.ceil((len(X_train)*0.9/32))
VAL_STEPS_PER_EPOCH = np.ceil((len(X_train)*0.1/8))
history = model2.fit(datagen.flow(X_train, y_train, batch_size=32,
         subset='training'),
         validation_data=datagen.flow(X_train, y_train,
         batch_size=8, subset='validation'),
         steps_per_epoch = TRAIN_STEPS_PER_EPOCH,
         validation_steps = VAL_STEPS_PER_EPOCH,
         epochs = 30, verbose=1, shuffle=True, 
         callbacks=callbacks_list)

C:\Users\RezaHardMan\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras_preprocessing\image\image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\RezaHardMan\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras_preprocessing\image\image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/30
1177/1182 [============================>.] - ETA: 0s - loss: 0.4677 - accuracy: 0.8486
Epoch 00001: val_accuracy improved from -inf to 0.96071, saving model to weights-improvement-01-0.96.hdf5

Epoch 00001: val_accuracy improved from -inf to 0.96071, saving model to best_weights.hdf5
1182/1182 [==============================] - 23s 12ms/step - loss: 0.4662 - accuracy: 0.8490 - val_loss: 0.1350 - val_accuracy: 0.9607
Epoch 2/30
1179/1182 [============================>.] - ETA: 0s - loss: 0.1231 - accuracy: 0.9628
Epoch 00002: val_accuracy improved from 0.96071 to 0.97714, saving model to weights-improvement-02-0.98.hdf5

Epoch 00002: val_accuracy improved from 0.96071 to 0.97714, saving model to best_weights.hdf5
1182/1182 [==============================] - 12s 11ms/step - loss: 0.1229 - accuracy: 0.9629 - val_loss: 0.0853 - val_accuracy: 0.9771
Epoch 3/30
1178/1182 [============================>.] - ETA: 0s - loss: 0.0880 - accuracy: 0.9746
Epoch 00003: val_accuracy improved

Epoch 23/30
1177/1182 [============================>.] - ETA: 0s - loss: 0.0198 - accuracy: 0.9946
Epoch 00023: val_accuracy did not improve from 0.99286

Epoch 00023: val_accuracy did not improve from 0.99286
1182/1182 [==============================] - 12s 11ms/step - loss: 0.0198 - accuracy: 0.9946 - val_loss: 0.0311 - val_accuracy: 0.9926
Epoch 24/30
1180/1182 [============================>.] - ETA: 0s - loss: 0.0212 - accuracy: 0.9939
Epoch 00024: val_accuracy did not improve from 0.99286

Epoch 00024: val_accuracy did not improve from 0.99286
1182/1182 [==============================] - 12s 10ms/step - loss: 0.0212 - accuracy: 0.9939 - val_loss: 0.0329 - val_accuracy: 0.9917
Epoch 25/30
1181/1182 [============================>.] - ETA: 0s - loss: 0.0198 - accuracy: 0.9944
Epoch 00025: val_accuracy did not improve from 0.99286

Epoch 00025: val_accuracy did not improve from 0.99286
1182/1182 [==============================] - 12s 10ms/step - loss: 0.0199 - accuracy: 0.9944 - val_l